<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

_By Jeff Hale - some adapted from Mahdi S_

# Time Series Forecasting Intro 🗓

## Learning Objectives
By the end of the lesson you will:

- Know how to approach a time series forecasting problem.
- Be able to make several baseline time series models.
- Make exponential smoothing models including Holt-Winters.

---

#### Prior knoweldge required
- Python, pandas, and machine learning basics
- Time series basics

---


## Common types of time series forecasting problems

- Forecast a single value of the time series. (use y historic values of y to predict y at one point in the future)
- Forecast a whole range of future values. (use historic values of y to predict y at many points in the future)
- Use other features to aid in your forecast. (predict many y in the future and use y and X)
- Forecast multiple time series simultaneously

## Approaching time series forecasting problems

## Before modeling
- Get the time period into *datetime* dtype format
- put it in the index
- sort the index

## Modeling considerations
- Make sure you are clear what you are predicting (one value, a range of values, one-step-ahead that updates every time period?)
- **Make sure you aren't leaking information from the future into your training data**
- Make null baseline models

## Example: Store sales data from Walmart

### Let's forecast one single value one time period into the future

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Prepare data

#### Read in the dataset, set the index as a datetime index, sort it, and take a look at it.

#### Rename sales column

#### Filter the DataFrame to Store 1 weekly sales only

Aggregate all departments to compute the total sales per store. Keep only that column.

Each department is in its own row, so you can aggregate sales by date.

#### Plot it

### Split the dataset into train and test sets.

Subset the data to train and test. **The test set must come after the training set.** Why?

#### Let's make the two datasets manually

Plot the two DataFrames

## Model 1: Naive Forecast

Take the last time period's sales and estimate the same value for all future time periods. This method is a **naive forecast**.

$${\Large \hat y_{t+1} = y_t}$$

Make a predictions DataFrame and set the predicted values equal to the last value in the `Sales` df.

#### Save our results in a *df_predictions* DataFrame.

#### Take the last training dataset value and make that the guess for all future time periods.

#### Plot the historic data, the actual "future" data, and the predicted "future" data.

#### Score our model
On RMSE.

#### That's the most basic baseline.


---
## Model 2: Simple average of the training data
Another baseline model. Maybe slightly better when the data don't have much of a trend.

#### Plot

#### Evaluate

## Model 3: Seasonality

Forecast = 52 observations prior if you have weekly sales data and the seasonal pattern repeats every 52 weeks.

---
## Model 4: Simple Exponential Smoothing

Smoothing just means using a moving average. Read more on moving averages from Hyndman [here](https://robjhyndman.com/papers/movingaverage.pdf).

Simple exponential smoothing uses weighted averages where the weights decrease exponentially for older data points. 

$${\Large\hat y_{t+1} = \alpha y_t + \alpha (1-\alpha)y_{t-1} + \alpha(1-\alpha)^2 y_{t-2} + ...}$$

The one-step-ahead forecast for time `t+1` is a weighted average of all of the observations in the series (`y1,…,yt`). The rate at which the weights decrease is controlled by the parameter, `α` (which is between 0 and 1).

**Older values matter less.**

If alpha =.5: 

- What is the weight of the most recent observation (t)? 
- What is the weight of (t-1)?
- What is the weight of (t-2)?

#### Let's use statsmodels for more advanced time series models to make life easier.

#### Summary info

#### The best thing about statsmodels is the summary. 

Here you get lots of info: AIC, BIC, AICC, etc. 🎉

The AIC (Akaike Information Criterion) and BIC (Bayesian Information Criterion) above are measures of how well the model fits. Lower is better. They consider the complexity of the model - like adjusted R2.

#### Forecast with `.forecast`

Here you can just pass an integer for the number of forecasts. 

#### Plot

You can play with the smoothing level, but SES forecasts the same number every time, so this isn't going to be a winner.

---
## Model 5: Holt-Winters' (additive and multiplicative versions)

- Part of the exponential smoothing family of algorithms. 
- It's also called *Triple Exponential Smoothing*

Holt-Winters' can handle treand and seasonality. It weights recent values more than old ones. This is a very strong classical time series model. 🎉

#### Params to choose: 
- number of seasonal_periods
- seasonal period effects additive or multiplicative
- additive or multiplicative trend

#### Plot

#### Evaluate

## Why did Holt-Winters' work well?

A time series dataset can be decomposed into its trend, seasonality, and residual components.

Let's break down the components using statsmodels `seasonal_decompose` function. 

Holt-Winters' uses exponential smoothing and takes into account the trend and errors seasonality to forecast future sales. 

You have to choose the number of seasonal periods, and whether the trend and seasonal components are additive or multiplicative. 

---
## Seasonality

Two primary kinds of seasonality:

### Additive vs. Multiplicative Seasonality

Seasonality can present itself in two ways:

1. __Additive:__ the peaks and valleys keep the same magnitude over time.
2. __Multiplicative:__ the peaks and valleys change in magnitude over time.

You can plot the data over time to see which kind of seasonality effects the data show:

#### Additive Seasonality
<img src="assets/adding.png" width="300px">

#### Multiplicative Seasonality
<img src="assets/multi.png" width="300px">

Alternatively, GridSearch it - or use a function that GridSearchs for you! 😀


We'll see some ways to deal with the trend in the next lesson. For now, just know that Hyndman says don't use multiplicative for trend. 

The **Holt-Winters** method takes into account both trend and seasonality to forecast future sales. 

With this method, we will apply exponential smoothing to the seasonal components as well as the level and trend components. 

Here are the constituent parts of the additive model:

<h3><center>Level Equation</center></h3> $$L_t = \alpha (y_t - S_{t-s}) + (1 - \alpha)(L_{t-1} + b_{t-1})$$ 
<h3><center>Trend Equation</center></h3> $$b_t = \beta *(L_t - L_{t-1}) + (1 - \beta)b_{t-1}$$
<h3><center>Seasonality Equation</center></h3> $$S_t = \gamma(y_t-L_t)+(1-\gamma)S_{t-s}$$

Combine these into a single equation: 

<h3><center>Forecast Equation</center></h3> $${\Large F_{t+k} = L_t + kb_t+S_{t+k-s}}$$
<br>

(Where `α`, `β`, and `γ` are the smoothing parameters — each between 0 and 1 — and `s` is the length of the seasonal cycle.)

* **The trend equation captures the overall direction of sales.** 
* The level equation is a weighted average between the seasonally adjusted observation and the non-seasonal forecast for time `t`. 
* The seasonal equation is a weighted average between the current seasonal index and the seasonal index of the same season `s` time periods ago. 

---
#### Write out the train, test, and prediction DataFrames to pickle file format to use later

In [ ]:
train.to_pickle('./train.pkl')
test.to_pickle('./test.pkl')
df_predictions.to_pickle('./df_predictions.pkl')

___

#### If my observations are by the minute and there is a big spike in users once per week, what number should I use for seasonality?

---
## Exercises

#### Hyperparameter search to see if a different Holt Winter's model will perform better.

#### Pick a different store. Does the model perform similarly? 

Note: Pandas has exponential weighted functions, rolling functions, and expanding window transformations. Great for looking at historical data, but statsmodels is far better for making forecasts.


---
## Check out the great free ebook: ⭐️
[Forecasting: Principles and Practice by
Rob J Hyndman & George Athanasopoulos](https://otexts.com/fpp3/) to learn more about time series. Hyndman is also behind a lot of the time series code and strategy in an R package that Python packages use.

Jason Brownlee also has a lot of great information and Python code on time series forecasting at [Machine Learning Mastery](https://machinelearningmastery.com/exponential-smoothing-for-time-series-forecasting-in-python/).

## Summary
- **Always sort your time series data by date**
- **Make sure you aren't leaking information from the future into your training data**
- Make baseline models first
- Consider the tradeoffs between complexity and performance when choosing which model to use
- Holt-Winters' Exponential Smoothing is really powerful and doesn't have too many parameters to choose
- Seasonality is how many observations it takes for a pattern to repeat

### Check for understanding

- How can you keep from leaking test data into your training data?
- What does *trend* mean?
- If you have data by the hour and a cycle repeats once every day, what will the seasonality be?
